### Initialization and general setup
This file is automatically generated using equi2py. Created on 2023-11-29 10:08:25



In [13]:
import os
from pathlib import Path
import numpy as np
from collections import OrderedDict

import chemapp as ca
from chemapp.friendly import (
    Info,
    StreamCalculation as casc,
    ThermochemicalSystem as cats,
    EquilibriumCalculation as caec,
    Units as cau,
    TemperatureUnit,
    PressureUnit,
    VolumeUnit,
    AmountUnit,
    EnergyUnit,
    Status,
)
from chemapp.core import (
    LimitVariable,
    ChemAppError,
)

### Load data file


In [14]:
cst_file = Path("HotMetalLiquid.cst")
# load the database file
cats.load(cst_file)

# set the limits for target calculations, if deviating from the standards as below:
# T = 250.0 - 10000.0 K
# P = 1e-35 - 1e08 bar
# V = 1e-08 - 1e35 dm3
# note that these limits are always set in the above units
ca.basic.tqclim(LimitVariable.TLOW, 100.0)

### Units setup
It is possible to change the units before retrieving results, or in between making multiple inputs. However, mixed

unit input is currently not generated automatically.



In [15]:
cau.set(
    T=TemperatureUnit.C,
    P=PressureUnit.atm,
    V=VolumeUnit.dm3,
    A=AmountUnit.tonne,
    E=EnergyUnit.J,
)

### Fixed conditions


In [16]:
# setting general boundary conditions of the calculation(s)
casc.set_eq_T(1310.0)
casc.set_eq_P(1.0)

### Stream setup
Create streams. The names are defaults of "#1", "#2", ...

Feel free to make these more reasonable, but take care to edit everywhere they are used



In [17]:
casc.create_st(name="#1", T=1300, P=1)
casc.create_st(name="#2", T=25, P=1)

### Variable conditions
define variables which are supposedly modified in iterations.



In [18]:
A = 0.0
A_range = [0.0]


# we keep the results in an OrderedDict, to access by variable value
# if wanted, the .values() attribute of results can be easily used like a list.
results = OrderedDict()

### Calculation


In [19]:
for A in A_range:
    # set incoming amounts
    # since this is a calculation with initial conditions, input needs to be made
    # using phase constituents
    casc.set_IA_pc("#1", "Fe_bcc(s)", "Fe_bcc(s)", 94.6225)
    casc.set_IA_pc("#1", "C_Graphite(s)", "C_Graphite(s)", 4.35)
    casc.set_IA_pc("#1", "Si_solid(s)", "Si_solid(s)", 0.505)
    casc.set_IA_pc("#1", "Mn_Solid_Alpha(s)", "Mn_Solid_Alpha(s)", 0.4)
    casc.set_IA_pc("#1", "P_Solid_(white)(s)", "P_Solid_(white)(s)", 0.1)
    casc.set_IA_pc("#1", "S_alpha_orthorhombic_(s)", "S_alpha_orthorhombic_(s)", 0.0225)
    casc.set_IA_pc("#2", "gas_ideal", "O2", A)
    try:
        # calculate
        casc.calculate_eq()
        # fetch results
        results[A] = casc.get_result_object()
    except ChemAppError as c_err:
        print(f"[<A> = { A }] ChemApp Error {c_err.errno}:")
        print(c_err)
        # change this if you want to capture a calculation error
        # maybe input a dummy value (or other, adequate replacements?):
        # results[A] = None
        pass

In [20]:
# this result listing is different from the Equilib one, but just in format.
print(results[0.0])

Stream Calculation Result
Class:                  chemapp.core.StreamCalculationResult
Calculator type:        builtins.NoneType
Calculation series ID:  None
Calculation start time: 2023-11-29 15:53:37
Calculation end time:   2023-11-29 15:53:37
Calculation duration:   0.0000E+00 s
Calculation inputs (from ChemApp tqshow):
 SYSTEM UNITS:
 Pressure     : atm
 Volume       : dm3
 Temperature  : C
 Energy       : J
 Amount       : tonne
 
 TARGET LIMITS:
 Pressure/bar : 1.00000E-50  1.00000E+07
 Volume/dm3   : 1.00000E-07  1.00000E+50
 Temperature/K:  100.00      6000.00

 T = 1310.00000000 C
 P = 1 atm

 STREAM NAME             STREAM NUMBER   TEMPERATURE/C   PRESSURE/atm
 #1                                                                           
                               1            1300.00       1.0000E+00
 #2                                                                           
                               2              25.00       1.0000E+00

 STREAM NUMBER STREAM CO

In [21]:
# quite a few properties are available as class attributes:
print("Calculated total enthalpy:", results[0.0].H)
print("Calculated amount of phase Fe-liq:", results[0.0].phs["Fe-liq"].A)
print("Calculated activity of Mn in Fe-liq:", results[0.0].phs["Fe-liq"].pcs["Mn"].AC)

Calculated total enthalpy: 123572064646.95676
Calculated amount of phase Fe-liq: 99.99999999999999
Calculated activity of Mn in Fe-liq: 0.0018924400150954024


In [22]:
import pandas as pd

# reading the csv file, setting the index to the index used in the csv file.
# remember to configure and convert units, if needed.
input_data = pd.read_csv("hotmetal.csv").set_index("ID")

In [23]:
# this will be displayed nicely in e.g. VSCode
input_data

,mass,T,Fe,C,Si,Mn,P,S
ID,,,,,,,,
0,97.09,1306.1,94.88,4.01,0.54,0.43,0.109,0.019
1,99.12,1361.6,94.71,4.28,0.51,0.39,0.096,0.020
2,94.30,1357.8,95.02,3.95,0.51,0.40,0.101,0.019
3,97.41,1340.3,94.96,3.99,0.52,0.41,0.095,0.018
4,95.73,1349.7,94.69,4.29,0.51,0.40,0.097,0.019
